In [1]:
%%capture
!pip install -q accelerate peft bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 25.1 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
%%capture
!pip install langchain faiss-cpu sentence-transformers
!pip install langchain_community

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/public-service/train_phogpt.jsonl
/kaggle/input/public-service/val_phogpt.jsonl
/kaggle/input/public-service/test_phogpt.jsonl


In [4]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [1]:
from datasets import load_dataset

dataset = load_dataset('json', data_files={
    "train": "/kaggle/input/public-service/train_phogpt.jsonl",
    "validation": "/kaggle/input/public-service/val_phogpt.jsonl",
    "test": "/kaggle/input/public-service/test_phogpt.jsonl"
}, split=None)

train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"]

print("Train size:", len(train_data))
print("Validation size:", len(val_data))
print("Test size:", len(test_data))

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Train size: 3517
Validation size: 100
Test size: 100


In [2]:
print(train_data[0])

{'instruction': 'Bạn là trợ lý dịch vụ công, trả lời ngắn gọn, chính xác.', 'input': 'Trung tâm hòa giải thương mại tự chấm dứt hoạt động thì cần thực hiện thủ tục gì tại Sở Tư pháp?', 'output': 'Theo quy định tại Điều 31 Nghị định số 22/2017/NĐ-CP ngày 24/02/2017 của Chính phủ về hòa giải thương mại thì Trung tâm hòa giải thương mại muốn tự chấm dứt hoạt động thì chậm nhất 30 ngày trước ngày chấm dứt hoạt động, Trung tâm hòa giải thương mại thông báo bằng văn bản về việc chấm dứt hoạt động cho Sở Tư pháp tỉnh, thành phố trực thuộc Trung ương nơi Trung tâm đăng ký hoạt động. Trong thời hạn 60 ngày, kể từ ngày quyết định chấm dứt hoạt động, Trung tâm hòa giải thương mại thanh toán xong các khoản nợ, nghĩa vụ tài sản khác, thanh lý các hợp đồng, hoàn tất các vụ việc đã nhận, trừ trường hợp có thỏa thuận khác.'}


In [6]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

2025-08-26 03:17:06.698294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756178227.077098      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756178227.186419      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
model_name = 'Viet-Mistral/Vistral-7B-Chat'
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
        device_map={"": 0},
        trust_remote_code=True,
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

('<s>', '</s>')

In [9]:
def merge_columns(examples):
    return {
        "text": [
            f"{i} {inp} {outp}"
            for i, inp, outp in zip(examples["instruction"], examples["input"], examples["output"])
        ]
    }

def tokenize_function(examples):
    enc = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    enc["labels"] = [l if l != tokenizer.pad_token_id else -100 for l in enc["input_ids"]]
    return enc


train_data = train_data.map(merge_columns, batched=True)
val_data = val_data.map(merge_columns, batched=True)
test_data = test_data.map(merge_columns, batched=True)

train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/3517 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/3517 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.05,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 46137344 || all params: 3850383360 || trainable%: 1.1982532565276824


In [11]:
training_args = TrainingArguments(
    output_dir="./Vistral-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=False,
    save_strategy="epoch",
    save_total_limit=3,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    warmup_ratio=0.03,
    weight_decay=0.001,
    lr_scheduler_type="constant"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [12]:
trainer.train()
trainer.save_model("./Vistral-finetuned-final")
tokenizer.save_pretrained("./Vistral-finetuned-final")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,2.274000
100,2.250000
150,2.381800
200,2.347200


('./Vistral-finetuned-final/tokenizer_config.json',
 './Vistral-finetuned-final/special_tokens_map.json',
 './Vistral-finetuned-final/chat_template.jinja',
 './Vistral-finetuned-final/tokenizer.model',
 './Vistral-finetuned-final/added_tokens.json',
 './Vistral-finetuned-final/tokenizer.json')

In [13]:
!zip -r Vistral-finetuned-final.zip ./Vistral-finetuned-final

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: Vistral-finetuned-final/ (stored 0%)
  adding: Vistral-finetuned-final/training_args.bin (deflated 51%)
  adding: Vistral-finetuned-final/special_tokens_map.json (deflated 70%)
  adding: Vistral-finetuned-final/chat_template.jinja (deflated 61%)
  adding: Vistral-finetuned-final/tokenizer.model (deflated 58%)
  adding: Vistral-finetuned-final/added_tokens.json (deflated 42%)
  adding: Vistral-finetuned-final/tokenizer.json (deflated 85%)
  adding: Vistral-finetuned-final/tokenizer_config.json (deflated 78%)
  adding: Vistral-finetuned-final/README.md (deflated 66%)
  adding: Vistral-finetuned-final/adapter_config.json (deflated 56%)
  adding: Vistral-finetuned-final/adapter_model.safetensors (deflated 59%)
